In [1]:
import urllib
import os
import zipfile
import pandas as pd
import tensorflow as tf

In [2]:
# This function downloads and extracts the dataset to the directory that contains this file.
# DO NOT CHANGE THIS CODE
# (unless you need to change the URL)
def download_and_extract_data():
    url = 'https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/household_power.zip'
    urllib.request.urlretrieve(url, 'household_power.zip')
    with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()


# This function normalizes the dataset using min max scaling.
# DO NOT CHANGE THIS CODE
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

# COMPLETE THE CODE IN THE FOLLOWING FUNCTION.
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    # YOUR CODE HERE
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(n_past + n_future, shift=shift, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch (n_past + n_future))
    dataset = dataset.shuffle(1000)
    dataset = dataset.map(lambda window: (window[:-n_future], window[-n_future:, :1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset # YOUR CODE HERE

# COMPLETE THE CODE IN THE FOLLOWING FUNCTION.
def solution_C5():
    # Downloads and extracts the dataset to the directory that contains this file.
    download_and_extract_data()
    # Reads the dataset from the csv.
    df = pd.read_csv('household_power_consumption.csv', sep=',',
                     infer_datetime_format=True, index_col='datetime', header=0)

    # Number of features in the dataset. We use all features as predictors to
    # predict all features at future time steps.
    N_FEATURES = df.shape[1] # YOUR CODE HERE

    # Normalizes the data
    # DO NOT CHANGE THIS
    data = df.values
    split_time = int(len(data) * 0.5)
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))

    # Splits the data into training and validation sets.
    x_train = data[:split_time] # YOUR CODE HERE
    x_valid = data[split_time:] # YOUR CODE HERE

    # DO NOT CHANGE THIS
    BATCH_SIZE = 32
    N_PAST = 24 # Number of past time steps based on which future observations should be predicted
    N_FUTURE = 24  # Number of future time steps which are to be predicted.
    SHIFT = 1  # By how many positions the window slides to create a new window of observations.

    # Code to create windowed train and validation datasets.
    # Complete the code in windowed_dataset.
    train_set = windowed_dataset(x_train, BATCH_SIZE, N_PAST, N_FUTURE, SHIFT) # YOUR CODE HERE
    valid_set = windowed_dataset(x_valid, BATCH_SIZE, N_PAST, N_FUTURE, SHIFT) # YOUR CODE HERE

    # Code to define your model.
    model = tf.keras.models.Sequential([
        # Whatever your first layer is, the input shape will be (N_PAST = 24, N_FEATURES = 7)
        # YOUR CODE HERE
        tf.keras.layers.LSTM(50, activation='relu', input_shape=[N_PAST, N_FEATURES]),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(N_FUTURE),
    ])

    # Code to train and compile the model
    # YOUR CODE HERE

    model.compile(optimizer=tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9), loss=tf.keras.losses.Huber(), metrics=['mae'])

    model.fit(train_set, validation_data=valid_set, epochs=10)

    return model

In [3]:
# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model = solution_C5()
    model.save("model_C5.h5")

C:\Users\ilham\AppData\Local\Temp\ipykernel_72104\825146727.py:34: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('household_power_consumption.csv', sep=',',


Epoch 1/10
1349/1349 [==============================] - 11s 7ms/step - loss: 0.0089 - mae: 0.1023 - val_loss: 0.0057 - val_mae: 0.0807
Epoch 2/10
1349/1349 [==============================] - 9s 7ms/step - loss: 0.0056 - mae: 0.0779 - val_loss: 0.0044 - val_mae: 0.0655
Epoch 3/10
1349/1349 [==============================] - 9s 7ms/step - loss: 0.0047 - mae: 0.0665 - val_loss: 0.0040 - val_mae: 0.0608
Epoch 4/10
1349/1349 [==============================] - 11s 8ms/step - loss: 0.0044 - mae: 0.0621 - val_loss: 0.0039 - val_mae: 0.0585
Epoch 5/10
1349/1349 [==============================] - 12s 9ms/step - loss: 0.0042 - mae: 0.0599 - val_loss: 0.0038 - val_mae: 0.0577
Epoch 6/10
1349/1349 [==============================] - 12s 9ms/step - loss: 0.0042 - mae: 0.0588 - val_loss: 0.0038 - val_mae: 0.0568
Epoch 7/10
1349/1349 [==============================] - 12s 8ms/step - loss: 0.0041 - mae: 0.0578 - val_loss: 0.0037 - val_mae: 0.0565
Epoch 8/10
1349/1349 [==============================] - 1

c:\Users\ilham\miniconda3\envs\vision1\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
